In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-18 00:40:41.847062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 00:40:42.383144: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-18 00:40:42.383189: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-18 00:40:42.383193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv



This error most likely means that this notebook is not configured to use a GPU.  Change this in Notebook Settings via the command palette (cmd/ctrl-shift-P) or the Edit menu.




SystemError: GPU device not found

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [28]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.1))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.4))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_reduced_3Conv.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=100, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 32, 32, 8)         224       
                                                                 
 max_pooling2d_86 (MaxPoolin  (None, 16, 16, 8)        0         
 g2D)                                                            
                                                                 
 dropout_94 (Dropout)        (None, 16, 16, 8)         0         
                                                                 
 conv2d_89 (Conv2D)          (None, 16, 16, 16)        1168      
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                            
                                                                 
 dropout_95 (Dropout)        (None, 8, 8, 16)        

Epoch 1/100
782/782 [==============================] - 8s 10ms/step - loss: 1.9659 - accuracy: 0.2607 - val_loss: 1.6349 - val_accuracy: 0.3969
Epoch 2/100
782/782 [==============================] - 7s 9ms/step - loss: 1.6577 - accuracy: 0.3857 - val_loss: 1.5392 - val_accuracy: 0.4481
Epoch 3/100
782/782 [==============================] - 7s 9ms/step - loss: 1.5466 - accuracy: 0.4298 - val_loss: 1.3587 - val_accuracy: 0.5122
Epoch 4/100
782/782 [==============================] - 7s 9ms/step - loss: 1.4800 - accuracy: 0.4610 - val_loss: 1.3277 - val_accuracy: 0.5314
Epoch 5/100
782/782 [==============================] - 7s 9ms/step - loss: 1.4258 - accuracy: 0.4797 - val_loss: 1.2613 - val_accuracy: 0.5371
Epoch 6/100
782/782 [==============================] - 7s 9ms/step - loss: 1.3926 - accuracy: 0.4973 - val_loss: 1.2298 - val_accuracy: 0.5568
Epoch 7/100
782/782 [==============================] - 7s 9ms/step - loss: 1.3563 - accuracy: 0.5072 - val_loss: 1.2304 - val_accuracy: 0.556

Epoch 58/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0542 - accuracy: 0.6290 - val_loss: 0.8950 - val_accuracy: 0.6884
Epoch 59/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0639 - accuracy: 0.6232 - val_loss: 0.8854 - val_accuracy: 0.6947
Epoch 60/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0598 - accuracy: 0.6266 - val_loss: 0.8916 - val_accuracy: 0.6941
Epoch 61/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0540 - accuracy: 0.6263 - val_loss: 0.9399 - val_accuracy: 0.6753
Epoch 62/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0564 - accuracy: 0.6266 - val_loss: 1.0181 - val_accuracy: 0.6504
Epoch 63/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0505 - accuracy: 0.6325 - val_loss: 0.8872 - val_accuracy: 0.6941
Epoch 64/100
782/782 [==============================] - 7s 9ms/step - loss: 1.0545 - accuracy: 0.6288 - val_loss: 0.9265 - val_accuracy:

In [29]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI_tiny_3Conv-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of reduced base quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmprscle91m/assets


INFO:tensorflow:Assets written to: /tmp/tmprscle91m/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-18 01:24:16.583844: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-18 01:24:16.583870: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-18 01:24:16.584412: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmprscle91m
2023-05-18 01:24:16.586387: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-18 01:24:16.586400: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmprscle91m
2023-05-18 01:24:16.592523: I tensorf

== Input details ==
name: serving_default_conv2d_88_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of reduced base quantized to int8 model is 69.81%


In [30]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

c_model_name = 'cifar10_base_tiny_3conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))